In [ ]:
# Polkadot Python Interface Testing Playbook

In [ ]:
!pip install substrate-interface
!pip install google-generativeai

In [9]:
# Converting textual datetime classified using token classification model, into Python usable format.

import google.generativeai as genai

genai.configure(api_key="KEY")

# Set up the model
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [30]:
# Obtaining usable DateTime format

from datetime import datetime

curr = datetime.now()

convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": ["The current date time is " + str(curr) + " . Convert text based unstructured date time into Python usable structured date time format considering dates in future from the obtained result from the API, don't give any code or additional text in the output, just give the formatted DateTime object"]
  },
  {
    "role": "model",
    "parts": ["​"]
  },
])

convo.send_message("Only give target datetime in python as the output - Send 10 DOT to Keval 48 hours from now")
print(convo.last.text)

datetime.datetime(2024, 3, 5, 18, 28, 53, 908113)


In [40]:
# On-Chain Query Balance Instruction
# RPC_URL = "wss://astar-rpc.dwellir.com"

from substrateinterface import SubstrateInterface

convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": ["Here's a sample url of a dwellir API endpoint which connects with a node of Aleph Zero network and enables us to make requests to the network - wss://aleph-zero-rpc.dwellir.com, for testnet the url becomes - wss://aleph-zero-testnet-rpc.dwellir.com, I will next provide you a prompt containing name of some other Polkadot ecosystem chain, I want you to only build the corresponding dwellir wss api for it. The output should be only the URL, do not give extra text or information."]
  },
  {
    "role": "model",
    "parts": ["​"]
  },
])

convo.send_message("Only give the url of the network - what is the balance of Bholu on Aleph Zero Testnet Network?")
print(convo.last.text)

substrate = SubstrateInterface(convo.last.text)

result = substrate.query('System', 'Account', ['5HMrodMT8vphQUyd3PxKE7i6G4rPLEqeZG18ener4Mvet1ei'])
print(result.value['data'])

wss://aleph-zero-testnet-rpc.dwellir.com
{'free': 25300989315289120, 'reserved': 0, 'frozen': 0, 'flags': 170141183460469231731687303715884105728}


In [4]:
# For Price Quote Fetch
# Use Gemini API for converting use requests into price feed api
# https://api.coinbase.com/v2/prices/BTC-INR/spot

import requests

convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": ["Here's a sample url of a coinbase API which fetches the price information of Bitcoin-USD pair (https://api.coinbase.com/v2/prices/BTC-INR/spot), I will next provide you a name of a cryptocurrency with its symbol or only a name, I want you to find the corresponding symbol for and construct the URL for its price feed on coinbase. The output that I expect is only URL and no extra text or information."]
  },
  {
    "role": "model",
    "parts": ["​"]
  },
])

convo.send_message("what is Ethereum's current price?")
print(convo.last.text)

response = requests.get(convo.last.text)
print(response.json())

https://api.coinbase.com/v2/prices/ETH-USD/spot
{'data': {'amount': '3434.26', 'base': 'ETH', 'currency': 'USD'}}


In [6]:
# Creating a new Keypair

from substrateinterface import Keypair

new_key = Keypair.generate_mnemonic()
print(new_key)
addr = Keypair.create_from_mnemonic(new_key)
print(addr)

rival genre matrix normal coconut glad wire carbon rookie degree stuff design
<Keypair (address=5FvCWaJGwkz1V7TTigKmD2uMfrZ6h75DAHpZUbKQMp9Q7tDK)>


In [62]:
# Creating a transfer extrinsic and calling it

from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException

seed_one = "rubber bachelor distance stem usual behave edit electric exile hungry budget marriage"
seed_two = "milk breeze flush panda carry acoustic sense potato intact prison thing confirm"

convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": ["You are being given a transaction prompt, from which you need to find out the amount that the user wants to send. Only give amount as output (for example - 10)."]
  },
  {
    "role": "model",
    "parts": ["​"]
  },
])

convo.send_message("Send 100 AZERO to Bholu from my wallet-1")
print(convo.last.text)

# Create a model chat that identifies network and decimals

substrate = SubstrateInterface("wss://aleph-zero-testnet-rpc.dwellir.com")

call = substrate.compose_call(
    call_module='Balances',
    call_function='transfer_allow_death',
    call_params={
        'dest': '5HMrodMT8vphQUyd3PxKE7i6G4rPLEqeZG18ener4Mvet1ei',
        'value': 1 * 10**12
    }
)

extrinsic = substrate.create_signed_extrinsic(call=call, keypair=Keypair.create_from_mnemonic(seed_one))

try:
    receipt = substrate.submit_extrinsic(extrinsic, wait_for_inclusion=True)

    print('Extrinsic "{}" included in block "{}"'.format(
        receipt.extrinsic_hash, receipt.block_hash
    ))

    if receipt.is_success:

        print('✅ Success, triggered events:')
        for event in receipt.triggered_events:
            print(f'* {event.value}')

    else:
        print('⚠️ Extrinsic Failed: ', receipt.error_message)


except SubstrateRequestException as e:
    print("Failed to send: {}".format(e))

100
Extrinsic "0x92415741ab543f9e0a15ff4045d562a7537b17ff6287ee282ff474a91042a036" included in block "0x0aab1ffb30188efd7c3a4192658411419f56f2de02e8b07bed32127e617541e7"
✅ Success, triggered events:
* {'phase': 'ApplyExtrinsic', 'extrinsic_idx': 1, 'event': {'event_index': '0508', 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': {'who': '5DSBAA9BEP5skMivJBye2J5dMLQVkRKVHmM5MJ8owfeNjmV9', 'amount': 308531145}}, 'event_index': 5, 'module_id': 'Balances', 'event_id': 'Withdraw', 'attributes': {'who': '5DSBAA9BEP5skMivJBye2J5dMLQVkRKVHmM5MJ8owfeNjmV9', 'amount': 308531145}, 'topics': []}
* {'phase': 'ApplyExtrinsic', 'extrinsic_idx': 1, 'event': {'event_index': '0502', 'module_id': 'Balances', 'event_id': 'Transfer', 'attributes': {'from': '5DSBAA9BEP5skMivJBye2J5dMLQVkRKVHmM5MJ8owfeNjmV9', 'to': '5HMrodMT8vphQUyd3PxKE7i6G4rPLEqeZG18ener4Mvet1ei', 'amount': 1000000000000}}, 'event_index': 5, 'module_id': 'Balances', 'event_id': 'Transfer', 'attributes': {'from': '5DSBAA9BEP5s